In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Read data for X and y
df7 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/7_countbased.csv")
df15 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/15_countbased.csv")
df16 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/16_countbased.csv")
df21 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/21_countbased.csv")
df33 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/33_countbased.csv")
df56 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/56_countbased.csv")
df68 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/68_countbased.csv")
df95 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/95_countbased.csv")
df1000 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/1000_countbased.csv")
df1001 = pd.read_csv("../Data/Train/Random Samples/10000/Count based/1001_countbased.csv")

df = pd.concat([df7,df15, df16, df21, df33, df56, df68, df95, df1000, df1001], ignore_index=True)

X = df.drop(columns=['y']).values
y = df['y'].values

In [56]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the dataset
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

#oversampler = RandomOverSampler(random_state=42)
#X_res, y_res = oversampler.fit_resample(X,y)

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.33)

In [24]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(criterion = "entropy", n_estimators = 50, max_features=9, min_samples_split=2, min_samples_leaf=3)
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

0.5509904622157007


In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
import joblib

# Define the scoring metric
scorer = make_scorer(f1_score)

model = RandomForestClassifier()

grid_space={'n_estimators':[10,20,30,40,50],
              'max_features':[1,3,5,7],
              'min_samples_leaf':[1,2,3],
              'min_samples_split':[1,2,3],
              'criterion': ["entropy","log_loss"]
           }

grid = GridSearchCV(model,param_grid=grid_space,cv=2,scoring="f1_micro",verbose=10)

# Enable verbose logging using joblib
#joblib.parallel_backend('threading')

model_grid = grid.fit(X_train,y_train)

Fitting 2 folds for each of 360 candidates, totalling 720 fits
[CV 1/2; 1/360] START criterion=entropy, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=10
[CV 1/2; 1/360] END criterion=entropy, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=10;, score=nan total time=   0.0s
[CV 2/2; 1/360] START criterion=entropy, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=10
[CV 2/2; 1/360] END criterion=entropy, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=10;, score=nan total time=   0.0s
[CV 1/2; 2/360] START criterion=entropy, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=20
[CV 1/2; 2/360] END criterion=entropy, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=20;, score=nan total time=   0.0s
[CV 2/2; 2/360] START criterion=entropy, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=20
[CV 2/2; 2/360] END criterion=entropy, max_features=

C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
240 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\ensemble\_forest.py", line 340, in fit
    self._validate_params()
  File "C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\base.py", line 600, in _validate_params
    valida

In [28]:
print('Best hyperparameters are: '+str(model_grid.best_params_))
print('Best score is: '+str(model_grid.best_score_))

Best hyperparameters are: {'criterion': 'log_loss', 'max_features': 7, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 40}
Best score is: 0.5581319812756371


In [12]:
y_pred = model.predict(X_test)

In [13]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           7       0.78      0.95      0.86      1634
          15       0.85      0.63      0.72       164
          16       0.94      0.67      0.78        88
          21       0.00      0.00      0.00        35
          33       0.38      0.09      0.14      1664
          56       0.87      0.39      0.54      1697
          68       0.38      0.69      0.49      1623
          95       0.00      0.00      0.00        88
        1000       0.72      0.67      0.69       910
        1001       0.38      0.57      0.46      1638

    accuracy                           0.54      9541
   macro avg       0.53      0.47      0.47      9541
weighted avg       0.58      0.54      0.52      9541



C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred, labels=model_grid.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=model.classes_)
disp.plot()
plt.show()

NameError: name 'model_grid' is not defined

In [17]:
#Verification with new unseen data
# Read the dataframe
df_verif7 = pd.read_csv("../Data/Test/3 months/Last 20/4 Separated by error type/Sequences/Classic/7.csv")
df_verif15 = pd.read_csv("../Data/Test/3 months/Last 20/4 Separated by error type/Sequences/Classic/15.csv")
df_verif16 = pd.read_csv("../Data/Test/3 months/Last 20/4 Separated by error type/Sequences/Classic/16.csv")
df_verif21 = pd.read_csv("../Data/Test/3 months/Last 20/4 Separated by error type/Sequences/Classic/21.csv")
df_verif33 = pd.read_csv("../Data/Test/3 months/Last 20/4 Separated by error type/Sequences/Classic/33.csv")
df_verif56 = pd.read_csv("../Data/Test/3 months/Last 20/4 Separated by error type/Sequences/Classic/56.csv")
df_verif68 = pd.read_csv("../Data/Test/3 months/Last 20/4 Separated by error type/Sequences/Classic/68.csv")
df_verif95 = pd.read_csv("../Data/Test/3 months/Last 20/4 Separated by error type/Sequences/Classic/95.csv")
df_verif1000 = pd.read_csv("../Data/Test/3 months/Last 20/4 Separated by error type/Sequences/Classic/1000.csv")
df_verif1001 = pd.read_csv("../Data/Test/3 months/Last 20/4 Separated by error type/Sequences/Classic/1001.csv")

# Drop any rows with missing values
df_verif = pd.concat([df_verif7, df_verif15, df_verif16, df_verif21, df_verif33, df_verif56, df_verif68, df_verif95, df_verif1000, df_verif1001], ignore_index=True)

# Split the data into input features (X) and target variable (y)
X_verif = df_verif.drop("y", axis=1).values
y_verif = df_verif["y"].values

In [18]:
y_pred_verif = model.predict(X_verif)
print(classification_report(y_verif, y_pred_verif))

              precision    recall  f1-score   support

           7       0.92      0.87      0.89       931
          15       0.00      0.00      0.00         5
          16       0.00      0.00      0.00         1
          21       0.00      0.00      0.00         6
          33       0.00      0.00      0.00         1
          56       0.54      0.14      0.23       554
          68       0.73      0.89      0.80     17408
          95       0.00      0.00      0.00        11
        1000       1.00      0.06      0.11        35
        1001       0.77      0.55      0.64     12389

    accuracy                           0.74     31341
   macro avg       0.40      0.25      0.27     31341
weighted avg       0.75      0.74      0.73     31341



C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
